<a href="https://colab.research.google.com/github/gshah8/UCF/blob/master/Machine_Learning/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Name: Ghayoor Shah

# HW 2

The goal of this homework is to create a convolutional neural network for the CIFAR10 data set. 
See [this colab notebook](https://colab.research.google.com/drive/1LZZviWOzvchcXRdZi2IBx3KOpQOzLalf) how to load the CIFAR data in Keras.

You should not use any pretrained convnets that come with Keras. You have to create and train your own convnets with Keras from scratch.

## Simple hold-out validation

Make sure that the data is divided into: 

- training set (80%)
- validation set (20%)
- test set. 

Use the training set to train your neural networks. Evaluate their performance on the validation data set. 

After trying several different architectures, choose the one that performs
best of the validation set. Try at least four different architectures by using data augmentation, using dropout, varying the number of layers, the number of filters, etc.

Train this final architecture on the data from the training set and validation set and evaluate its performance on 
the test set.

## k-fold validation

Reevaluate your best architecture using k-fold validation with k=5, that is, the size of the validation fold is 20%. Does the accuracy/loss obtain by k-fold validation differ from the accuracy/loss obtain by simple hold-out validation.

### Loading the CIFAR10 data set



In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.datasets import cifar10

(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

Using TensorFlow backend.


170500096/170498071 [==============================] - 10s 0us/step


### Exploring the format of the CIFAR10 data set

In [2]:
train_images.shape

(50000, 32, 32, 3)

In [3]:
train_images.ndim

4

In [4]:
train_labels.shape

(50000, 1)

In [5]:
train_labels.ndim

2

###Using Simple Hold Validation for the models below

In [0]:
#divide training data into training and validation data
rand_idx = np.random.permutation(len(train_images))
val_idx = rand_idx[0:2000]
train_idx = rand_idx[2000:]

train_images_new, train_labels_new = train_images[train_idx] , train_labels[train_idx]
val_images, val_labels = train_images[val_idx] , train_labels[val_idx]

#normalize training and validation data
train_images_norm = train_images_new/255
val_images_norm = val_images/255

#train_images_norm.shape
#train_images_norm.shape


###Preprocess test data

In [0]:
#train_images_norm = train_images/255
test_images_norm = test_images/255

###Build a basic model (without Data Augmentation and Dropout)

In [8]:
# set up the layers

model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    #
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    #
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    #
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dens

###Compile the Model

In [0]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

###Train the Model

In [10]:
epochs = 20
history = model.fit(train_images_norm, 
                      train_labels_new, 
                      epochs=epochs,  
                      validation_data=(val_images_norm, val_labels))

Train on 48000 samples, validate on 2000 samples
Epoch 1/20
48000/48000 [==============================] - 18s 376us/sample - loss: 1.5674 - acc: 0.4284 - val_loss: 1.3371 - val_acc: 0.5290
Epoch 2/20
48000/48000 [==============================] - 16s 327us/sample - loss: 1.2041 - acc: 0.5723 - val_loss: 1.2887 - val_acc: 0.5475
Epoch 3/20
48000/48000 [==============================] - 16s 343us/sample - loss: 1.0498 - acc: 0.6285 - val_loss: 1.1383 - val_acc: 0.6055
Epoch 4/20
48000/48000 [==============================] - 16s 335us/sample - loss: 0.9401 - acc: 0.6723 - val_loss: 1.0160 - val_acc: 0.6425
Epoch 5/20
48000/48000 [==============================] - 14s 297us/sample - loss: 0.8672 - acc: 0.6968 - val_loss: 1.0261 - val_acc: 0.6455
Epoch 6/20
48000/48000 [==============================] - 14s 290us/sample - loss: 0.8016 - acc: 0.7193 - val_loss: 0.9890 - val_acc: 0.6540
Epoch 7/20
48000/48000 [==============================] - 14s 301us/sample - loss: 0.7512 - acc: 0.7359 -

We can observe that the model is clearly overfitted since the training accuracy is increasing as we increase the number of epochs but the testing accuracy is not increasing.

Now, we will try to improve the model by using the four following architectures:
1. increasing the number of convolutional layers
2. Adding more filters
3. Data Augmentation
4. Dropout

We will add the above mentioned architectures one by one and observe the train and validation accuracy.
In the end, as a sanity check, we will run the model for test data.